In [ ]:
# import statements
from time import sleep
from json import dumps
from kafka import KafkaProducer
import datetime as dt
import pandas as pd
import random
from datetime import timezone,datetime 

In [ ]:
#Taken from tutorial
def publish_message(producer_instance, topic_name, value):
    try:
        producer_instance.send(topic_name, value=value)
#         producer_instance.flush()
#         print('Message published successfully. Data: ' + str(value))
#         print('########################################################')
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  value_serializer=lambda x: dumps(x).encode('ascii'),
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer

In [ ]:
#Function to send an attack per machine using dictionary to keep track of indexes
def send_attack_per_machine(df_machine,start_index_dict,end_index_dict,prevDataY):
    #Which machine to send rows from
    key = df_machine.machine[0]
    #Random number of rows to send
    XrowsToSend = random.randrange(10,50)
    YrowsToSend = random.randrange(1,5)
    #Size of the given df
    totalRows = int(df_machine.shape[0])
#     print('Machine No: %d'%key)
#     print('Rows to send: %d'%(XrowsToSend))
    #Add to the end index of the selected machine 
    end_index_dict[key] = end_index_dict[key]+XrowsToSend
    #Get the start index of the machine from previous run
    start_index = start_index_dict[key]
    #Get end index from previous run
    end_index = end_index_dict[key]
    #Check whether end index is lesser than total number of rows in df
    if(end_index>totalRows):
        print('second')
        #Set start_index of the particular machine to 0 in the dict
        start_index_dict[key] = 0
        #Get the start_index which is now 0
        start_index = start_index_dict[key]
        #update the end_index for the particular machine to rowsToSend in the dictt
        end_index_dict[key] = XrowsToSend
        #get the end index
        end_index = end_index_dict[key]
#         print('end index: %d'%(end_index))
#         print('start index: %d'%(start_index))
        #Create a temp df with the selected rows to be sent
    tempX = df_machine.iloc[start_index:end_index,:]
    tempY = df_machine.iloc[start_index:(start_index+YrowsToSend),:]
    unixTimeStamp = int(datetime.now().replace(tzinfo=timezone.utc).timestamp())
    #appened the ts in unix-timestamp
    tempX['ts'] = unixTimeStamp
    tempY['ts'] = unixTimeStamp
    #publish the message to kafka stream
#     print(prevDataY)
    #Check for first time sending only the generated X rows
    if not prevDataY:
        publish_message(producer, topic, tempX.to_dict(orient='record'))
    else:
        publish_message(producer, topic, tempX.to_dict(orient='record'))
        #publish previous data
        publish_message(producer, topic, prevDataY)
    #update the start_index to end_index
    start_index_dict[key] = end_index
#     print('start_index_dict :%s'%(str(start_index_dict)))
#     print('end_index_dict :%s'%(str(end_index_dict)))
    #Return data to be published next cycle
    return tempY.to_dict(orient='record')

In [ ]:
start_index_dict = {4:0, 5:0, 6:0, 7:0, 8:0}
end_index_dict = {4:0, 5:0, 6:1, 7:0, 8:0}
prev_data_dict = {4:[], 5:[], 6:[], 7:[], 8:[]}
producer = connect_kafka_producer()
topic = 'Assignment2B-memory'
df_memory = pd.read_csv('Streaming_Linux_memory.csv')
df_machine_4 =  df_memory.loc[df_memory['machine']==4].reset_index()
df_machine_5 = df_memory.loc[df_memory['machine']==5].reset_index()
df_machine_6 = df_memory.loc[df_memory['machine']==6].reset_index()
df_machine_7 = df_memory.loc[df_memory['machine']==7].reset_index()
df_machine_8 = df_memory.loc[df_memory['machine']==8].reset_index()

In [ ]:
# df_memory.groupby('machine').count()

In [ ]:
while True:
    prev_data_dict[4]=send_attack_per_machine(df_machine_4,start_index_dict,end_index_dict,prev_data_dict[4])
    prev_data_dict[5]=send_attack_per_machine(df_machine_5,start_index_dict,end_index_dict,prev_data_dict[5])
    prev_data_dict[6]=send_attack_per_machine(df_machine_6,start_index_dict,end_index_dict,prev_data_dict[6])
    prev_data_dict[7]=send_attack_per_machine(df_machine_7,start_index_dict,end_index_dict,prev_data_dict[7])
    prev_data_dict[8]=send_attack_per_machine(df_machine_8,start_index_dict,end_index_dict,prev_data_dict[8])
    sleep(10)